# OS Maps Example

OS Maps Example in Python. Adapted from the tutorial [price-paid-spatial-distribution](https://github.com/OrdnanceSurvey/os-data-hub-tutorials/tree/master/data-science/price-paid-spatial-distribution) by Ordnance Survey.

### Step 0: Install requirements

If you are using this notebook directly from GitHub, all the requirements should be in `requirements.txt`. So you can just install the requirements using the provided `make` job:

```shell
make requirements
```

### Step 1: Import Python libraries

In [ ]:
import fiona
import folium
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from folium.plugins import FloatImage
from mpl_toolkits.axes_grid1 import make_axes_locatable
from zipfile import ZipFile